## Problem Set 2


### Problem 1

Clean up the file. This means getting rid of duplicates; you can assume that no student can register for the same course more than once. How many duplicate records do you find? Some of the fields have bad or missing values; repair those that you can (and explain what a repair means).

### Import Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import hashlib
from functools import reduce

In [ ]:
NA_FILL_VALUE = 0

In [ ]:
columnNames = []
with open('dirty_sample_small_header.csv', 'r') as headerFile:
    headerReader = csv.reader(headerFile, delimiter=',')
    for row in headerReader:
        columnNames.append(row[1])
        
numCols = len(columnNames)

In [ ]:
invalidCols = 0; duplicateRows = 0; keptRows = 0; totalRows = 0
onHeader = True
hashes = set()
with open('dirty_sample_small.csv', 'r') as dataFile:
    with open('valid_rows_sample_small.csv', 'w') as outFile:
        dataReader = csv.reader(dataFile, delimiter=',')
        outWriter = csv.writer(outFile, delimiter = ',')
        for row in dataReader:
            # Skip the header line
            if onHeader:
                outWriter.writerow(row)
                onHeader = False; continue
        
            totalRows += 1
            # Ignore rows with incorrect number of columns
            if len(row) != numCols:
                invalidCols += 1
                continue 
            # Get the md5 hash of each row to determine whether the row is duplicated or not
            else:
                m = hashlib.md5()
                rowStr = reduce((lambda x, y: x + y), row).encode('utf-8')
                m.update(rowStr)
                hashedRow = m.hexdigest()
                if hashedRow in hashes:
                    duplicateRows += 1
                    continue 
                # If it's a new row, write it to the cleaned dataset valid_rows...
                else:            
                    keptRows += 1
                    hashes.add(hashedRow)
                    outWriter.writerow(row)
            # Ignore rows where there are fewer entries than half the number of columns
            count = 0
            for word in row:
                if word == "nan":
                    count += 1
            if count <= int(numCols/2):
                continue 
print("Dropped: %d   Duplicates: %d   Kept: %d   Total: %d" % (invalidCols, duplicateRows, keptRows, totalRows))

In [ ]:
invalidCols + duplicateRows + keptRows == totalRows

In [ ]:
df_test = pd.read_csv("valid_rows_sample_small.csv", sep=',', engine='python', error_bad_lines=False, dtype='unicode')

In [ ]:
# Use Pandas drop_duplicates() as evidence that dataset is deduplicated
print("Deduplicated Valid Rows: %d\tFully Deduplicated: %r" 
      % (len(df_test), len(df_test) == len(df_test.drop_duplicates())))

In [ ]:
len(df_test.columns.values)

In [ ]:
df_test.head(10)

Some fields may have values that are incompatible types. This may occur when no data is stored for a variable, a user did not complete the course or course registration, or a column may contain multiple data types. A string representation of an age cannot be compared to a number. If a user inputted N/A, or left that field blank, it is interpreted differently as NA, na, NaN.

In order to repair bad or missing values, we must understand which columns these values come from, which type all the data in that column should be represented with, and how empty values should be coded.

In [ ]:
df_test.replace("nan", np.nan, inplace=True)
df_test.replace("None", np.nan, inplace=True)
nullCount = df_test.columns[df_test.isnull().any()]
df_test[nullCount].isnull().sum()


### Problem 2

Some lines may be corrupt; get rid of those or mark them in some way to show that they are not good lines. How many corrupt lines are there? Does the count of corrupt lines change if you get rid of them before getting rid of the duplicate records? What difference might this make to the remaining data set?

A line with no user id or course number is corrupt. The count of corrupt lines does not change if we get rid of them before or after the duplicate records. 

### Problem 3

What are some possible sources of bias in this data set? Is there anything unusual about the data set that you should flag?

Students who registered more than once for the same class, or reigstered for the same class under different names, and students who enrolled but did not participate in a class to any extent, are possible sources of bias in this data set. The dataset has an enormous number of duplicate rows, and many unreliable birth dates. Any self-reported data regarding a student's demographics will bias the data set.